In [2]:
#author: akarmaka
#Reading the h5 hourly EPMEDIUM load file found in inputs/loaddata to disaggregate down to county level

import gzip
import pandas as pd
import h5py


df_p1_census = pd.read_csv('co-est2021-alldata.csv', header=0, usecols=['STNAME','CTYNAME','POPESTIMATE2021','STATE','COUNTY'], nrows=3195, encoding='latin-1')
#print(df_p1_census)

df_p1_counties = pd.read_excel('county_reeds_corrected0310.xlsx', header=0, usecols=['NAME','STATE_NAME','FIPS','PCA_REG'])
#print(df_p1_counties)


def read_file(filename, index_columns=1):
    """
    Read input file of various types (for backwards-compatibility)
    """
    # Try reading a .h5 file written by pandas
    try:
        df = pd.read_hdf(filename+'.h5')
    # Try reading a .h5 file written by h5py
    except (ValueError, TypeError, FileNotFoundError, OSError):
        try:
            with h5py.File(filename+'.h5', 'r') as f:
                keys = list(f)
                datakey = 'data' if 'data' in keys else ('cf' if 'cf' in keys else 'load')
                ### If none of these keys work, we're dealing with EER-formatted load
                if datakey not in keys:
                    years = [int(y) for y in keys if y != 'columns']
                    df = pd.concat(
                        {y: pd.DataFrame(f[str(y)][...]) for y in years},
                        axis=0)
                    df.index = df.index.rename(['year','hour'])
                else:
                    df = pd.DataFrame(f[datakey][:])
                    df.index = pd.Series(f['index']).values
                df.columns = pd.Series(f['columns']).map(
                    lambda x: x if type(x) is str else x.decode('utf-8')).values
        # Fall back to .csv.gz
        except (FileNotFoundError, OSError):
            df = pd.read_csv(
                filename+'.csv.gz', index_col=list(range(index_columns)),
                float_precision='round_trip',
            )

    return df


df_p1load_1week = read_file('D://akash//reeds//ReEDS-2.0//inputs//loaddata//EPMEDIUM_load_hourly',
            index_columns=2)





#Below is the dataframe containing load information as in the heirarchy of the file - years and then hours

df_p1load_1week

p1           p10        p100          p101          p102  \
year hour                                                                       
2010 0      5324.812012  11578.527344  282.707184  10493.142578   5894.619141   
     1      5151.875977  11063.793945  274.441071   9966.186523   5523.636719   
     2      5017.917969  10666.641602  269.360046   9724.698242   5297.368164   
     3      4944.829590  10389.592773  267.740967   9665.220703   5188.648926   
     4      4982.326660  10276.935547  271.241425   9859.899414   5204.319824   
...                 ...           ...         ...           ...           ...   
2050 8755  16427.582031  30556.191406  706.747925  30959.199219  14899.032227   
     8756  15431.430664  28747.121094  677.350891  29246.738281  13853.796875   
     8757  11557.686523  21847.847656  643.989807  26871.955078  12549.620117   
     8758  10885.644531  20855.378906  602.285461  24230.132812  11025.229492   
     8759  10348.269531  19848.939453  558.269104  21683.128906   9620.510742   

                   p103         p104          p105         p106         p107  \
year hour                                                                      
2010 0      8321.033203   901.878784   6318.256836  1648.038330  2604.994629   
     1      7978.739258   882.480774   6209.529297  1610.189331  2545.515137   
     2      7669.498047   864.553467   6091.838379  1567.385986  2480.086426   
     3      7465.434570   854.570801   5993.365723  1524.143066  2414.016357   
     4      7371.051270   855.040649   6013.633301  1523.775391  2414.323242   
...                 ...          ...           ...          ...          ...   
2050 8755  17549.644531  1890.645630  13489.460938  3430.040771  5444.672852   
     8756  16897.685547  1833.129395  13137.981445  3343.332764  5305.294434   
     8757  15855.120117  1739.810669  12518.619141  3184.513672  5053.120605   
     8758  14708.878906  1631.869385  11870.595703  3042.606934  4824.363770   
     8759  13278.595703  1505.178223  11049.988281  2835.360107  4495.750977   

           ...           p90          p91           p92          p93  \
year hour  ...                                                         
2010 0     ...   3709.161621   909.666382   9678.579102  1518.546875   
     1     ...   3581.422363   873.684753   9480.302734  1488.976929   
     2     ...   3518.217529   864.174316   9293.382812  1454.768433   
     3     ...   3506.709961   858.344971   9233.838867  1446.048584   
     4     ...   3513.884033   879.064209   9313.854492  1445.335693   
...        ...           ...          ...           ...          ...   
2050 8755  ...  10363.132812  2875.861816  22939.695312  3606.206299   
     8756  ...   9999.809570  2755.620605  21991.259766  3460.661133   
     8757  ...   9468.723633  2581.343262  20848.392578  3288.996582   
     8758  ...   8730.265625  2354.751953  19588.242188  3102.978516   
     8759  ...   5636.894043  1432.365356  14218.607422  2247.393555   

                    p94          p95           p96           p97  \
year hour                                                          
2010 0     12926.119141  2490.752686   5564.055176   6407.276367   
     1     12727.002930  2488.581055   5579.327148   6398.021973   
     2     12674.793945  2511.077393   5645.944336   6456.051270   
     3     12688.828125  2560.292236   5779.888672   6581.703613   
     4     12860.987305  2651.829590   6006.695312   6812.997070   
...                 ...          ...           ...           ...   
2050 8755  32541.066406  5950.766113  12966.903320  15587.134766   
     8756  31999.246094  5705.337402  12599.126953  14939.485352   
     8757  30470.669922  5397.713867  12109.096680  14149.102539   
     8758  28681.667969  5082.298828  11609.925781  13334.923828   
     8759  26776.433594  4737.017578  11094.939453  12433.081055   

                    p98           p99  
year hour                              
2010 0      6262.463867 

In [3]:
#Checking datatypes
df_p1load_1week.dtypes

p1      float32
p10     float32
p100    float32
p101    float32
p102    float32
         ...   
p95     float32
p96     float32
p97     float32
p98     float32
p99     float32
Length: 134, dtype: object

In [4]:
#Removing 'p' from BA regions
df_p1load_1week.columns = df_p1load_1week.columns.str.replace('p', '')
df_p1load_1week

1            10         100           101           102  \
year hour                                                                       
2010 0      5324.812012  11578.527344  282.707184  10493.142578   5894.619141   
     1      5151.875977  11063.793945  274.441071   9966.186523   5523.636719   
     2      5017.917969  10666.641602  269.360046   9724.698242   5297.368164   
     3      4944.829590  10389.592773  267.740967   9665.220703   5188.648926   
     4      4982.326660  10276.935547  271.241425   9859.899414   5204.319824   
...                 ...           ...         ...           ...           ...   
2050 8755  16427.582031  30556.191406  706.747925  30959.199219  14899.032227   
     8756  15431.430664  28747.121094  677.350891  29246.738281  13853.796875   
     8757  11557.686523  21847.847656  643.989807  26871.955078  12549.620117   
     8758  10885.644531  20855.378906  602.285461  24230.132812  11025.229492   
     8759  10348.269531  19848.939453  558.269104  21683.128906   9620.510742   

                    103          104           105          106          107  \
year hour                                                                      
2010 0      8321.033203   901.878784   6318.256836  1648.038330  2604.994629   
     1      7978.739258   882.480774   6209.529297  1610.189331  2545.515137   
     2      7669.498047   864.553467   6091.838379  1567.385986  2480.086426   
     3      7465.434570   854.570801   5993.365723  1524.143066  2414.016357   
     4      7371.051270   855.040649   6013.633301  1523.775391  2414.323242   
...                 ...          ...           ...          ...          ...   
2050 8755  17549.644531  1890.645630  13489.460938  3430.040771  5444.672852   
     8756  16897.685547  1833.129395  13137.981445  3343.332764  5305.294434   
     8757  15855.120117  1739.810669  12518.619141  3184.513672  5053.120605   
     8758  14708.878906  1631.869385  11870.595703  3042.606934  4824.363770   
     8759  13278.595703  1505.178223  11049.988281  2835.360107  4495.750977   

           ...            90           91            92           93  \
year hour  ...                                                         
2010 0     ...   3709.161621   909.666382   9678.579102  1518.546875   
     1     ...   3581.422363   873.684753   9480.302734  1488.976929   
     2     ...   3518.217529   864.174316   9293.382812  1454.768433   
     3     ...   3506.709961   858.344971   9233.838867  1446.048584   
     4     ...   3513.884033   879.064209   9313.854492  1445.335693   
...        ...           ...          ...           ...          ...   
2050 8755  ...  10363.132812  2875.861816  22939.695312  3606.206299   
     8756  ...   9999.809570  2755.620605  21991.259766  3460.661133   
     8757  ...   9468.723633  2581.343262  20848.392578  3288.996582   
     8758  ...   8730.265625  2354.751953  19588.242188  3102.978516   
     8759  ...   5636.894043  1432.365356  14218.607422  2247.393555   

                     94           95            96            97  \
year hour                                                          
2010 0     12926.119141  2490.752686   5564.055176   6407.276367   
     1     12727.002930  2488.581055   5579.327148   6398.021973   
     2     12674.793945  2511.077393   5645.944336   6456.051270   
     3     12688.828125  2560.292236   5779.888672   6581.703613   
     4     12860.987305  2651.829590   6006.695312   6812.997070   
...                 ...          ...           ...           ...   
2050 8755  32541.066406  5950.766113  12966.903320  15587.134766   
     8756  31999.246094  5705.337402  12599.126953  14939.485352   
     8757  30470.669922  5397.713867  12109.096680  14149.102539   
     8758  28681.667969  5082.298828  11609.925781  13334.923828   
     8759  26776.433594  4737.017578  11094.939453  12433.081055   

                     98            99  
year hour                              
2010 0      6262.463867  

In [5]:
#Transposing the file to have BA regions as rows in 1 single column
#Doing so also maintains the original indexes intact as we will need them back later
t_df_p1load_1week = df_p1load_1week.transpose()
t_df_p1load_1week

year          2010                                                          \
hour          0             1             2             3             4      
1      5324.812012   5151.875977   5017.917969   4944.829590   4982.326660   
10    11578.527344  11063.793945  10666.641602  10389.592773  10276.935547   
100     282.707184    274.441071    269.360046    267.740967    271.241425   
101   10493.142578   9966.186523   9724.698242   9665.220703   9859.899414   
102    5894.619141   5523.636719   5297.368164   5188.648926   5204.319824   
..             ...           ...           ...           ...           ...   
95     2490.752686   2488.581055   2511.077393   2560.292236   2651.829590   
96     5564.055176   5579.327148   5645.944336   5779.888672   6006.695312   
97     6407.276367   6398.021973   6456.051270   6581.703613   6812.997070   
98     6262.463867   6247.486328   6339.109375   6505.345215   6749.589844   
99     8701.742188   8621.164062   8541.605469   8637.170898   8791.201172   

year                                                                        \
hour          5             6             7             8             9      
1      5118.870117   5375.929688   5627.222656   5828.738770   5988.346191   
10    10340.834961  10525.402344  10484.271484  10807.454102  11111.666992   
100     278.914612    289.131989    300.235992    308.200989    311.483978   
101   10320.019531  10761.982422  11804.114258  12545.936523  12982.857422   
102    5352.123535   5516.114258   6145.168457   6845.809570   7415.694824   
..             ...           ...           ...           ...           ...   
95     2769.436035   2878.248291   2959.234375   2892.006836   2737.086914   
96     6235.083496   6484.539062   6649.541016   6334.160156   5907.404785   
97     7117.550781   7397.731445   7600.857422   7418.019043   7013.244629   
98     7106.835449   7429.043945   7587.577148   7237.948730   6707.538086   
99     9153.129883   9617.997070   9938.009766  10096.303711   9750.555664   

year  ...          2050                                            \
hour  ...          8750          8751          8752          8753   
1     ...  14130.305664  14423.455078  15434.859375  16219.142578   
10    ...  23988.996094  24570.964844  26169.990234  29239.031250   
100   ...    617.563477    622.646545    651.590698    690.970032   
101   ...  23563.806641  24426.072266  27249.529297  30018.804688   
102   ...  12475.859375  12743.180664  13954.344727  15109.291016   
..    ...           ...           ...           ...           ...   
95    ...   4532.131348   4712.169434   5345.823730   5644.228027   
96    ...   9405.785156   9806.416992  11180.869141  11954.199219   
97    ...  11867.039062  12338.116211  14022.998047  14798.794922   
98    ...  11851.470703  12404.704102  14310.110352  15089.699219   
99    ...  17145.402344  17175.572266  18324.818359  20014.603516   

year                                                                        \
hour          8754          8755          8756          8757          8758   
1     16691.099609  16427.582031  15431.430664  11557.686523  10885.644531   
10    31031.636719  30556.191406  28747.121094  21847.847656  20855.378906   
100     716.875549    706.747925    677.350891    643.989807    602.285461   
101   30586.263672  30959.199219  29246.738281  26871.955078  24230.132812   
102   15127.300781  14899.032227  13853.796875  12549.620117  11025.229492   
..             ...           ...           ...           ...           ...   
95     5890.974609   5950.766113   5705.337402   5397.713867   5082.298828   
96    12675.413086  12966.903320  12599.126953  12109.096680  11609.925781   
97    15449.231445  15587.134766  14939.485352  14149.102539  13334.923828   
98    15861.364258  16019.871094  15406.906250  14613.060547  13771.171875   
99    21161.132812  21102.246094  20520.724609  19499.068359  18478.474609   

year                
hour          8759  
1     10348.269

In [6]:
# Zero padding data in State and County codes present in Population estimates file to match Reeds County file format

df_p1_census['STATE']=df_p1_census['STATE'].apply(lambda x: '{0:0>2}'.format(x))
df_p1_census['COUNTY']=df_p1_census['COUNTY'].apply(lambda x: '{0:0>3}'.format(x))
df_p1_census.sample()

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2021
861,19,109,Iowa,Kossuth County,14529


In [7]:
# Creating new column for entire FIPS code needed to join this dataframe to Population dataframe

df_p1_census['FIPS']=df_p1_census['STATE'] + df_p1_census['COUNTY']
df_p1_census.sample(15)
#df_p1_census.dtypes

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2021,FIPS
1964,37,077,North Carolina,Granville County,61986,37077
1851,35,041,New Mexico,Roosevelt County,19019,35041
1101,21,177,Kentucky,Muhlenberg County,30694,21177
2251,41,007,Oregon,Clatsop County,41810,41007
68,02,000,Alaska,Alaska,732673,02000
872,19,131,Iowa,Mitchell County,10555,19131
325,10,000,Delaware,Delaware,1003384,10000
2118,39,075,Ohio,Holmes County,44271,39075
1858,35,055,New Mexico,Taos County,34623,35055
2797,48,457,Texas,Tyler County,20077,48457


In [8]:
# Ensuring datatype consistency for created column.

df_p1_census['FIPS']= df_p1_census['FIPS'].apply(pd.to_numeric)
df_p1_census.dtypes

STATE              object
COUNTY             object
STNAME             object
CTYNAME            object
POPESTIMATE2021     int64
FIPS                int64
dtype: object

In [9]:
# Inner join between Population dataframe and Reeds Counties dataframe

df_census_counties = pd.merge(df_p1_census, df_p1_counties, on='FIPS', how='inner')
#, suffixes=('_censustbl','_countiestbl'))
#df_census_counties.sample()
df_census_counties

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2021,FIPS,NAME,STATE_NAME,PCA_REG
0,01,001,Alabama,Autauga County,59095,1001,Autauga,Alabama,90
1,01,003,Alabama,Baldwin County,239294,1003,Baldwin,Alabama,90
2,01,005,Alabama,Barbour County,24964,1005,Barbour,Alabama,90
3,01,007,Alabama,Bibb County,22477,1007,Bibb,Alabama,89
4,01,009,Alabama,Blount County,59041,1009,Blount,Alabama,90
...,...,...,...,...,...,...,...,...,...
3104,56,037,Wyoming,Sweetwater County,41614,56037,Sweetwater,Wyoming,21
3105,56,039,Wyoming,Teton County,23575,56039,Teton,Wyoming,21
3106,56,041,Wyoming,Uinta County,20635,56041,Uinta,Wyoming,21
3107,56,043,Wyoming,Washakie County,7705,56043,Washakie,Wyoming,21


In [10]:
# To get aggregate of BA region population, group by BA region number (PCA_REG column) and get sum of populations for
# each set of counties inside respective BA region

#df_groupedbyPCA = df_census_counties.groupby('PCA_REG')
df_groupedbyPCA = df_census_counties.groupby(['PCA_REG'])['POPESTIMATE2021'].sum().reset_index(name='County_sum_pop')
df_groupedbyPCA.sample(10)
#df_groupedbyPCA.info()

,PCA_REG,County_sum_pop
2,3,835253
129,130,1388992
88,89,2550763
44,45,696537
120,121,531203
105,106,1327752
68,69,320192
33,34,1344554
7,8,85938
66,67,6321308


In [11]:
# Full Outer join between above dataframe which contains aggregates of populations with dataframe containing county names, FIPS
# and county-wise population (going to be required to get the proportional factor in next steps)

df_popbycounty = pd.merge(df_groupedbyPCA, df_census_counties, on='PCA_REG', how='outer')
df_popbycounty.head(20)

,PCA_REG,County_sum_pop,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2021,FIPS,NAME,STATE_NAME
0,1,5174505,53,007,Washington,Chelan County,79646,53007,Chelan,Washington
1,1,5174505,53,029,Washington,Island County,87432,53029,Island,Washington
2,1,5174505,53,033,Washington,King County,2252305,53033,King,Washington
3,1,5174505,53,035,Washington,Kitsap County,274314,53035,Kitsap,Washington
4,1,5174505,53,037,Washington,Kittitas County,45499,53037,Kittitas,Washington
5,1,5174505,53,053,Washington,Pierce County,925708,53053,Pierce,Washington
6,1,5174505,53,055,Washington,San Juan County,18557,53055,San Juan,Washington
7,1,5174505,53,057,Washington,Skagit County,130696,53057,Skagit,Washington
8,1,5174505,53,061,Washington,Snohomish County,833540,53061,Snohomish,Washington
9,1,5174505,53,067,Washington,Thurston County,297977,53067,Thurston,Washington


In [12]:
# Dividing each county's population by the cumulative sum of it's respective BA Region

df_popbycounty['proportion_ofpop']=df_popbycounty['POPESTIMATE2021']/df_popbycounty['County_sum_pop']
#df_popbycounty.head(20)



In [13]:
# Changing load dataframe index datatype to int for ease of operations

t_df_p1load_1week.index = t_df_p1load_1week.index.astype(int)


In [14]:
t_df_p1load_1week.index

Int64Index([  1,  10, 100, 101, 102, 103, 104, 105, 106, 107,
            ...
             90,  91,  92,  93,  94,  95,  96,  97,  98,  99],
           dtype='int64', length=134)

In [15]:
# Outer join between dataframe containing proportional factors for each county and df containing transposed load data
#The join should yield all counties under BA regions of interest and as per the county_Reeds_corrected file.

df_popbycounty_value = pd.merge(df_popbycounty, t_df_p1load_1week, left_on='PCA_REG', how='inner', right_index=True) #
#df_popbycounty_value.head(20)

#Replacing NaN values with 0s if present
import numpy as np

clean_df_popbycounty_value = df_popbycounty_value.replace(np.nan,0)
print(clean_df_popbycounty_value)

C:\Users\akarmaka\AppData\Local\Temp\27\ipykernel_18552\3872568862.py:4: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df_popbycounty_value = pd.merge(df_popbycounty, t_df_p1load_1week, left_on='PCA_REG', how='inner', right_index=True) #


      PCA_REG  County_sum_pop STATE COUNTY      STNAME            CTYNAME  \
0           1         5174505    53    007  Washington      Chelan County   
1           1         5174505    53    029  Washington      Island County   
2           1         5174505    53    033  Washington        King County   
3           1         5174505    53    035  Washington      Kitsap County   
4           1         5174505    53    037  Washington    Kittitas County   
...       ...             ...   ...    ...         ...                ...   
3104      134         1372247    23    023       Maine   Sagadahoc County   
3105      134         1372247    23    025       Maine    Somerset County   
3106      134         1372247    23    027       Maine       Waldo County   
3107      134         1372247    23    029       Maine  Washington County   
3108      134         1372247    23    031       Maine        York County   

      POPESTIMATE2021   FIPS        NAME  STATE_NAME  ...  (2050, 8750)  \


In [16]:
# Dropping columns from df that are not required

clean_df_popbycounty_value.drop(['COUNTY', 'STATE','STNAME','NAME'], axis=1, inplace=True)
clean_df_popbycounty_value


,PCA_REG,County_sum_pop,CTYNAME,POPESTIMATE2021,FIPS,STATE_NAME,proportion_ofpop,"(2010, 0)","(2010, 1)","(2010, 2)",...,"(2050, 8750)","(2050, 8751)","(2050, 8752)","(2050, 8753)","(2050, 8754)","(2050, 8755)","(2050, 8756)","(2050, 8757)","(2050, 8758)","(2050, 8759)"
0,1,5174505,Chelan County,79646,53007,Washington,0.015392,5324.812012,5151.875977,5017.917969,...,14130.305664,14423.455078,15434.859375,16219.142578,16691.099609,16427.582031,15431.430664,11557.686523,10885.644531,10348.269531
1,1,5174505,Island County,87432,53029,Washington,0.016897,5324.812012,5151.875977,5017.917969,...,14130.305664,14423.455078,15434.859375,16219.142578,16691.099609,16427.582031,15431.430664,11557.686523,10885.644531,10348.269531
2,1,5174505,King County,2252305,53033,Washington,0.435270,5324.812012,5151.875977,5017.917969,...,14130.305664,14423.455078,15434.859375,16219.142578,16691.099609,16427.582031,15431.430664,11557.686523,10885.644531,10348.269531
3,1,5174505,Kitsap County,274314,53035,Washington,0.053013,5324.812012,5151.875977,5017.917969,...,14130.305664,14423.455078,15434.859375,16219.142578,16691.099609,16427.582031,15431.430664,11557.686523,10885.644531,10348.269531
4,1,5174505,Kittitas County,45499,53037,Washington,0.008793,5324.812012,5151.875977,5017.917969,...,14130.305664,14423.455078,15434.859375,16219.142578,16691.099609,16427.582031,15431.430664,11557.686523,10885.644531,10348.269531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,134,1372247,Sagadahoc County,37071,23023,Maine,0.027015,1152.312500,1093.940552,1044.254395,...,2292.723145,2294.326416,2332.165527,2511.001953,2691.991943,2643.218506,2483.744629,2346.185059,2136.251709,1917.231201
3105,134,1372247,Somerset County,50592,23025,Maine,0.036868,1152.312500,1093.940552,1044.254395,...,2292.723145,2294.326416,2332.165527,2511.001953,2691.991943,2643.218506,2483.744629,2346.185059,2136.251709,1917.231201
3106,134,1372247,Waldo County,39912,23027,Maine,0.029085,1152.312500,1093.940552,1044.254395,...,2292.723145,2294.326416,2332.165527,2511.001953,2691.991943,2643.218506,2483.744629,2346.185059,2136.251709,1917.231201
3107,134,1372247,Washington County,31121,23029,Maine,0.022679,1152.312500,1093.940552,1044.254395,...,2292.723145,2294.326416,2332.165527,2511.001953,2691.991943,2643.218506,2483.744629,2346.185059,2136.251709,1917.231201


In [17]:
# Multiplying the proportional factors column with each load value column. Earlier join ensures
# we have counties by BA Region number (PCA_REG column). This is the disaggregation by weight.

df_newload = clean_df_popbycounty_value.iloc[:, 7:].multiply(clean_df_popbycounty_value['proportion_ofpop'], axis="index")
df_newload

,"(2010, 0)","(2010, 1)","(2010, 2)","(2010, 3)","(2010, 4)","(2010, 5)","(2010, 6)","(2010, 7)","(2010, 8)","(2010, 9)",...,"(2050, 8750)","(2050, 8751)","(2050, 8752)","(2050, 8753)","(2050, 8754)","(2050, 8755)","(2050, 8756)","(2050, 8757)","(2050, 8758)","(2050, 8759)"
0,81.959526,79.297694,77.235812,76.110835,76.687990,78.789668,82.746330,86.614232,89.715969,92.172647,...,217.493717,222.005874,237.573412,249.645102,256.909467,252.853403,237.520638,177.895953,167.551881,159.280603
1,89.971691,87.049644,84.786198,83.551246,84.184822,86.491955,90.835410,95.081429,98.486384,101.183221,...,238.755376,243.708630,260.798014,274.049803,282.024314,277.571739,260.740080,195.286631,183.931347,174.851489
2,2317.729081,2242.455273,2184.147417,2152.334264,2168.655601,2228.088824,2339.979054,2449.359258,2537.073106,2606.545374,...,6150.493255,6278.092299,6718.325897,7059.700575,7265.129149,7150.427944,6716.833483,5030.710212,4738.190727,4504.287696
3,282.282167,273.114376,266.012913,262.138308,264.126125,271.364650,284.992048,298.313743,308.996638,317.457843,...,749.084341,764.624956,818.242134,859.819031,884.838704,870.868950,818.060369,612.703094,577.076396,548.588746
4,46.820637,45.300025,44.122143,43.479483,43.809192,45.009807,47.270111,49.479709,51.251624,52.655039,...,124.246624,126.824263,135.717458,142.613597,146.763476,144.446388,135.687310,101.625794,95.716584,90.991489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,31.129510,29.552603,28.210340,27.366386,27.131969,27.160418,27.746125,29.068301,30.541047,32.631240,...,61.937494,61.980806,63.003022,67.834255,72.723667,71.406061,67.097904,63.381757,57.710447,51.793648
3105,42.483455,40.331398,38.499569,37.347798,37.027880,37.066706,37.866040,39.670457,41.680361,44.532914,...,84.528113,84.587222,85.982275,92.575616,99.248354,97.450175,91.570693,86.499147,78.759324,70.684477
3106,33.515174,31.817417,30.372288,29.463656,29.211274,29.241903,29.872497,31.296001,32.881613,35.131991,...,66.684180,66.730812,67.831367,73.032850,78.296970,76.878388,72.240067,68.239128,62.133186,55.762943
3107,26.133136,24.809327,23.682501,22.974004,22.777211,22.801094,23.292794,24.402757,25.639123,27.393834,...,51.996351,52.032712,52.890860,56.946666,61.051313,59.945187,56.328501,53.208807,48.447757,43.480621


In [18]:
#joining older dataframe containing all data with values of newly disaggregated data.
df_newload_joinedtoorig = pd.merge(clean_df_popbycounty_value, df_newload, left_index=True, right_index=True)
df_newload_joinedtoorig


,PCA_REG,County_sum_pop,CTYNAME,POPESTIMATE2021,FIPS,STATE_NAME,proportion_ofpop,"(2010, 0)_x","(2010, 1)_x","(2010, 2)_x",...,"(2050, 8750)_y","(2050, 8751)_y","(2050, 8752)_y","(2050, 8753)_y","(2050, 8754)_y","(2050, 8755)_y","(2050, 8756)_y","(2050, 8757)_y","(2050, 8758)_y","(2050, 8759)_y"
0,1,5174505,Chelan County,79646,53007,Washington,0.015392,5324.812012,5151.875977,5017.917969,...,217.493717,222.005874,237.573412,249.645102,256.909467,252.853403,237.520638,177.895953,167.551881,159.280603
1,1,5174505,Island County,87432,53029,Washington,0.016897,5324.812012,5151.875977,5017.917969,...,238.755376,243.708630,260.798014,274.049803,282.024314,277.571739,260.740080,195.286631,183.931347,174.851489
2,1,5174505,King County,2252305,53033,Washington,0.435270,5324.812012,5151.875977,5017.917969,...,6150.493255,6278.092299,6718.325897,7059.700575,7265.129149,7150.427944,6716.833483,5030.710212,4738.190727,4504.287696
3,1,5174505,Kitsap County,274314,53035,Washington,0.053013,5324.812012,5151.875977,5017.917969,...,749.084341,764.624956,818.242134,859.819031,884.838704,870.868950,818.060369,612.703094,577.076396,548.588746
4,1,5174505,Kittitas County,45499,53037,Washington,0.008793,5324.812012,5151.875977,5017.917969,...,124.246624,126.824263,135.717458,142.613597,146.763476,144.446388,135.687310,101.625794,95.716584,90.991489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,134,1372247,Sagadahoc County,37071,23023,Maine,0.027015,1152.312500,1093.940552,1044.254395,...,61.937494,61.980806,63.003022,67.834255,72.723667,71.406061,67.097904,63.381757,57.710447,51.793648
3105,134,1372247,Somerset County,50592,23025,Maine,0.036868,1152.312500,1093.940552,1044.254395,...,84.528113,84.587222,85.982275,92.575616,99.248354,97.450175,91.570693,86.499147,78.759324,70.684477
3106,134,1372247,Waldo County,39912,23027,Maine,0.029085,1152.312500,1093.940552,1044.254395,...,66.684180,66.730812,67.831367,73.032850,78.296970,76.878388,72.240067,68.239128,62.133186,55.762943
3107,134,1372247,Washington County,31121,23029,Maine,0.022679,1152.312500,1093.940552,1044.254395,...,51.996351,52.032712,52.890860,56.946666,61.051313,59.945187,56.328501,53.208807,48.447757,43.480621


In [19]:
# Dropping columns from df that are not required

df_newload_joinedtoorig.drop(['PCA_REG', 'County_sum_pop','CTYNAME','POPESTIMATE2021', 'STATE_NAME', 'proportion_ofpop'], axis=1, inplace=True)


In [20]:
#Adding line to ensure FIPS code is uniform with leading zeros
df_newload_joinedtoorig['FIPS']=df_newload_joinedtoorig['FIPS'].apply(lambda x: '{0:0>5}'.format(x))
df_newload_joinedtoorig.sample(20)

,FIPS,"(2010, 0)_x","(2010, 1)_x","(2010, 2)_x","(2010, 3)_x","(2010, 4)_x","(2010, 5)_x","(2010, 6)_x","(2010, 7)_x","(2010, 8)_x",...,"(2050, 8750)_y","(2050, 8751)_y","(2050, 8752)_y","(2050, 8753)_y","(2050, 8754)_y","(2050, 8755)_y","(2050, 8756)_y","(2050, 8757)_y","(2050, 8758)_y","(2050, 8759)_y"
2234,37069,6262.463867,6247.486328,6339.109375,6505.345215,6749.589844,7106.835449,7429.043945,7587.577148,7237.948730,...,162.728321,170.324571,196.487025,207.191282,217.786740,219.963141,211.546739,200.646727,189.087053,177.813658
2557,18043,2604.994629,2545.515137,2480.086426,2414.016357,2414.323242,2410.291748,2458.368164,2495.857910,2534.551270,...,199.011602,201.019060,209.577809,220.784807,232.407848,232.995105,227.030653,216.239322,206.450080,192.387679
800,40011,3229.661377,3186.604980,3158.427246,3159.675049,3200.094238,3273.254395,3370.438232,3482.066650,3554.834229,...,21.203110,21.429160,22.347663,23.201760,24.249690,23.901450,22.796337,21.528576,20.100682,17.464033
1003,29211,2095.720703,2057.611572,2040.403687,2055.375000,2101.540527,2171.716553,2248.512939,2298.412354,2296.285400,...,15.276707,15.702692,16.792106,17.835475,18.431299,18.171106,17.250739,16.282348,15.100286,13.231250
1867,47043,9678.579102,9480.302734,9293.382812,9233.838867,9313.854492,9482.902344,9726.735352,10061.734375,10063.777344,...,150.987449,163.484085,178.342463,183.033404,186.523216,181.841146,174.322972,165.263555,155.274443,112.709774
369,08007,1592.894287,1595.217285,1667.293701,1652.045532,1642.807983,1631.569092,1627.620850,1589.444946,1606.167114,...,37.728651,38.516557,40.773404,42.046609,43.560268,43.251155,41.034309,38.932819,31.324317,30.414911
689,27117,162.700363,158.974625,156.355728,155.691330,156.694977,160.218948,165.598724,170.865845,175.181717,...,22.533308,22.957102,24.389520,25.540960,26.362608,26.130615,24.722044,23.205778,21.488984,16.376545
2208,37171,6407.276367,6398.021973,6456.051270,6581.703613,6812.997070,7117.550781,7397.731445,7600.857422,7418.019043,...,158.445509,164.735204,187.231292,197.589523,206.273977,208.115225,199.467984,188.915006,178.044311,166.003149
120,06113,10460.324219,9978.130859,9632.873047,9406.898438,9356.245117,9481.457031,9734.548828,9811.309570,10091.121094,...,298.173301,306.853820,330.128500,369.424481,390.778877,383.839434,360.376622,273.217478,260.311957,247.282538
39,41003,3707.505371,3584.514404,3500.452881,3463.711426,3494.872070,3609.465576,3785.487549,3953.179688,4110.551270,...,244.090811,248.884975,267.837143,287.831545,294.677450,287.214079,272.220739,216.010152,204.306523,193.686123


In [21]:
#Renaming FIPS column as per initial file
df_newload_joinedtoorig.rename(columns = {'FIPS':'r'}, inplace = True)
#df_newload_joinedtoorig

#Reindexing or changing datatypes is for ensuring correct joins and operations
#df_newload_joinedtoorig = df_newload_joinedtoorig.reindex(['i','r','timeslice','value'], axis=1)
df_newload_joinedtoorig = df_newload_joinedtoorig.reindex(axis=1)
#df_newload_joinedtoorig

#Adding the "p" back in fips column
df_newload_joinedtoorig['r'] = 'p' + df_newload_joinedtoorig['r'].astype(str)
df_newload_joinedtoorig

,r,"(2010, 0)_x","(2010, 1)_x","(2010, 2)_x","(2010, 3)_x","(2010, 4)_x","(2010, 5)_x","(2010, 6)_x","(2010, 7)_x","(2010, 8)_x",...,"(2050, 8750)_y","(2050, 8751)_y","(2050, 8752)_y","(2050, 8753)_y","(2050, 8754)_y","(2050, 8755)_y","(2050, 8756)_y","(2050, 8757)_y","(2050, 8758)_y","(2050, 8759)_y"
0,p53007,5324.812012,5151.875977,5017.917969,4944.829590,4982.326660,5118.870117,5375.929688,5627.222656,5828.738770,...,217.493717,222.005874,237.573412,249.645102,256.909467,252.853403,237.520638,177.895953,167.551881,159.280603
1,p53029,5324.812012,5151.875977,5017.917969,4944.829590,4982.326660,5118.870117,5375.929688,5627.222656,5828.738770,...,238.755376,243.708630,260.798014,274.049803,282.024314,277.571739,260.740080,195.286631,183.931347,174.851489
2,p53033,5324.812012,5151.875977,5017.917969,4944.829590,4982.326660,5118.870117,5375.929688,5627.222656,5828.738770,...,6150.493255,6278.092299,6718.325897,7059.700575,7265.129149,7150.427944,6716.833483,5030.710212,4738.190727,4504.287696
3,p53035,5324.812012,5151.875977,5017.917969,4944.829590,4982.326660,5118.870117,5375.929688,5627.222656,5828.738770,...,749.084341,764.624956,818.242134,859.819031,884.838704,870.868950,818.060369,612.703094,577.076396,548.588746
4,p53037,5324.812012,5151.875977,5017.917969,4944.829590,4982.326660,5118.870117,5375.929688,5627.222656,5828.738770,...,124.246624,126.824263,135.717458,142.613597,146.763476,144.446388,135.687310,101.625794,95.716584,90.991489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,p23023,1152.312500,1093.940552,1044.254395,1013.013977,1004.336609,1005.389709,1027.070679,1076.013306,1130.529541,...,61.937494,61.980806,63.003022,67.834255,72.723667,71.406061,67.097904,63.381757,57.710447,51.793648
3105,p23025,1152.312500,1093.940552,1044.254395,1013.013977,1004.336609,1005.389709,1027.070679,1076.013306,1130.529541,...,84.528113,84.587222,85.982275,92.575616,99.248354,97.450175,91.570693,86.499147,78.759324,70.684477
3106,p23027,1152.312500,1093.940552,1044.254395,1013.013977,1004.336609,1005.389709,1027.070679,1076.013306,1130.529541,...,66.684180,66.730812,67.831367,73.032850,78.296970,76.878388,72.240067,68.239128,62.133186,55.762943
3107,p23029,1152.312500,1093.940552,1044.254395,1013.013977,1004.336609,1005.389709,1027.070679,1076.013306,1130.529541,...,51.996351,52.032712,52.890860,56.946666,61.051313,59.945187,56.328501,53.208807,48.447757,43.480621


In [22]:
#Checking length of FIPS column. All should be of length 6.
len_check = df_newload_joinedtoorig['r'].str.len()==5
len_check

0       False
1       False
2       False
3       False
4       False
        ...  
3104    False
3105    False
3106    False
3107    False
3108    False
Name: r, Length: 3109, dtype: bool

In [23]:
#Droppping duplicate columns with older values of load
df_newload_joinedtoorig.drop(df_newload_joinedtoorig.iloc[:, 1:359161], axis=1, inplace=True)

#View dataframe and some samples of FIPS length
df_newload_joinedtoorig

#Cleaning up column names after join caused X & Y additions in names
df_newload_joinedtoorig.columns = df_newload_joinedtoorig.columns.str.replace("_y", "")
print(df_newload_joinedtoorig)

           r    (2010, 0)    (2010, 1)    (2010, 2)    (2010, 3)    (2010, 4)  \
0     p53007    81.959526    79.297694    77.235812    76.110835    76.687990   
1     p53029    89.971691    87.049644    84.786198    83.551246    84.184822   
2     p53033  2317.729081  2242.455273  2184.147417  2152.334264  2168.655601   
3     p53035   282.282167   273.114376   266.012913   262.138308   264.126125   
4     p53037    46.820637    45.300025    44.122143    43.479483    43.809192   
...      ...          ...          ...          ...          ...          ...   
3104  p23023    31.129510    29.552603    28.210340    27.366386    27.131969   
3105  p23025    42.483455    40.331398    38.499569    37.347798    37.027880   
3106  p23027    33.515174    31.817417    30.372288    29.463656    29.211274   
3107  p23029    26.133136    24.809327    23.682501    22.974004    22.777211   
3108  p23031   180.197801   171.069638   163.299752   158.414398   157.057437   

        (2010, 5)    (2010,

In [24]:
df_newload_joinedtoorig.dtypes

r                object
(2010, 0)       float64
(2010, 1)       float64
(2010, 2)       float64
(2010, 3)       float64
                 ...   
(2050, 8755)    float64
(2050, 8756)    float64
(2050, 8757)    float64
(2050, 8758)    float64
(2050, 8759)    float64
Length: 359161, dtype: object

In [25]:
df_newload_joinedtoorig

,r,"(2010, 0)","(2010, 1)","(2010, 2)","(2010, 3)","(2010, 4)","(2010, 5)","(2010, 6)","(2010, 7)","(2010, 8)",...,"(2050, 8750)","(2050, 8751)","(2050, 8752)","(2050, 8753)","(2050, 8754)","(2050, 8755)","(2050, 8756)","(2050, 8757)","(2050, 8758)","(2050, 8759)"
0,p53007,81.959526,79.297694,77.235812,76.110835,76.687990,78.789668,82.746330,86.614232,89.715969,...,217.493717,222.005874,237.573412,249.645102,256.909467,252.853403,237.520638,177.895953,167.551881,159.280603
1,p53029,89.971691,87.049644,84.786198,83.551246,84.184822,86.491955,90.835410,95.081429,98.486384,...,238.755376,243.708630,260.798014,274.049803,282.024314,277.571739,260.740080,195.286631,183.931347,174.851489
2,p53033,2317.729081,2242.455273,2184.147417,2152.334264,2168.655601,2228.088824,2339.979054,2449.359258,2537.073106,...,6150.493255,6278.092299,6718.325897,7059.700575,7265.129149,7150.427944,6716.833483,5030.710212,4738.190727,4504.287696
3,p53035,282.282167,273.114376,266.012913,262.138308,264.126125,271.364650,284.992048,298.313743,308.996638,...,749.084341,764.624956,818.242134,859.819031,884.838704,870.868950,818.060369,612.703094,577.076396,548.588746
4,p53037,46.820637,45.300025,44.122143,43.479483,43.809192,45.009807,47.270111,49.479709,51.251624,...,124.246624,126.824263,135.717458,142.613597,146.763476,144.446388,135.687310,101.625794,95.716584,90.991489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,p23023,31.129510,29.552603,28.210340,27.366386,27.131969,27.160418,27.746125,29.068301,30.541047,...,61.937494,61.980806,63.003022,67.834255,72.723667,71.406061,67.097904,63.381757,57.710447,51.793648
3105,p23025,42.483455,40.331398,38.499569,37.347798,37.027880,37.066706,37.866040,39.670457,41.680361,...,84.528113,84.587222,85.982275,92.575616,99.248354,97.450175,91.570693,86.499147,78.759324,70.684477
3106,p23027,33.515174,31.817417,30.372288,29.463656,29.211274,29.241903,29.872497,31.296001,32.881613,...,66.684180,66.730812,67.831367,73.032850,78.296970,76.878388,72.240067,68.239128,62.133186,55.762943
3107,p23029,26.133136,24.809327,23.682501,22.974004,22.777211,22.801094,23.292794,24.402757,25.639123,...,51.996351,52.032712,52.890860,56.946666,61.051313,59.945187,56.328501,53.208807,48.447757,43.480621


In [26]:
#Getting rid of index name 'r' because there is non in original load file
df_newload_joinedtoorig.rename(columns = {'r':''}, inplace = True)
df_newload_joinedtoorig

,,"(2010, 0)","(2010, 1)","(2010, 2)","(2010, 3)","(2010, 4)","(2010, 5)","(2010, 6)","(2010, 7)","(2010, 8)",...,"(2050, 8750)","(2050, 8751)","(2050, 8752)","(2050, 8753)","(2050, 8754)","(2050, 8755)","(2050, 8756)","(2050, 8757)","(2050, 8758)","(2050, 8759)"
0,p53007,81.959526,79.297694,77.235812,76.110835,76.687990,78.789668,82.746330,86.614232,89.715969,...,217.493717,222.005874,237.573412,249.645102,256.909467,252.853403,237.520638,177.895953,167.551881,159.280603
1,p53029,89.971691,87.049644,84.786198,83.551246,84.184822,86.491955,90.835410,95.081429,98.486384,...,238.755376,243.708630,260.798014,274.049803,282.024314,277.571739,260.740080,195.286631,183.931347,174.851489
2,p53033,2317.729081,2242.455273,2184.147417,2152.334264,2168.655601,2228.088824,2339.979054,2449.359258,2537.073106,...,6150.493255,6278.092299,6718.325897,7059.700575,7265.129149,7150.427944,6716.833483,5030.710212,4738.190727,4504.287696
3,p53035,282.282167,273.114376,266.012913,262.138308,264.126125,271.364650,284.992048,298.313743,308.996638,...,749.084341,764.624956,818.242134,859.819031,884.838704,870.868950,818.060369,612.703094,577.076396,548.588746
4,p53037,46.820637,45.300025,44.122143,43.479483,43.809192,45.009807,47.270111,49.479709,51.251624,...,124.246624,126.824263,135.717458,142.613597,146.763476,144.446388,135.687310,101.625794,95.716584,90.991489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,p23023,31.129510,29.552603,28.210340,27.366386,27.131969,27.160418,27.746125,29.068301,30.541047,...,61.937494,61.980806,63.003022,67.834255,72.723667,71.406061,67.097904,63.381757,57.710447,51.793648
3105,p23025,42.483455,40.331398,38.499569,37.347798,37.027880,37.066706,37.866040,39.670457,41.680361,...,84.528113,84.587222,85.982275,92.575616,99.248354,97.450175,91.570693,86.499147,78.759324,70.684477
3106,p23027,33.515174,31.817417,30.372288,29.463656,29.211274,29.241903,29.872497,31.296001,32.881613,...,66.684180,66.730812,67.831367,73.032850,78.296970,76.878388,72.240067,68.239128,62.133186,55.762943
3107,p23029,26.133136,24.809327,23.682501,22.974004,22.777211,22.801094,23.292794,24.402757,25.639123,...,51.996351,52.032712,52.890860,56.946666,61.051313,59.945187,56.328501,53.208807,48.447757,43.480621


In [27]:
# # Cleaning Load data - Renaming Columns from format "p(BARegion)" to just BARegion number
# # Transposing the entire dataframe so now we have all BA region numbers as 1 column and subsequent columns are hourly timesteps.
# # Now the load df is 135 rows (1 for each BA region (1st row is just headers)) by 61320 columns (7 years worth of hourly data)

# df_p1load_1week.columns = df_p1load_1week.columns.str.replace("p", "")
# print(df_p1load_1week)

# df_p1load_1week.rename(columns = {"": "PCA_REG"}, inplace=True)
# print(df_p1load_1week)

# df_p1load_1week_transposed = df_p1load_1week.transpose()
# df_p1load_1week_transposed


In [28]:
#Transpose the dataframe to go back to original form of hourly and yearly data and BA regions as columns
df_original_form = df_newload_joinedtoorig.transpose()
df_original_form

,0,1,2,3,4,5,6,7,8,9,...,3099,3100,3101,3102,3103,3104,3105,3106,3107,3108
,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,p53067,...,p23013,p23015,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031
"(2010, 0)",81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,306.63252,...,34.499333,30.08573,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801
"(2010, 1)",79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,296.673894,...,32.751723,28.561696,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638
"(2010, 2)",77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,288.959841,...,31.264158,27.26444,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752
"(2010, 3)",76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,284.751003,...,30.328845,26.448784,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2050, 8755)",252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,945.992247,...,79.135891,69.011798,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611
"(2050, 8756)",237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,888.628268,...,74.361368,64.848094,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201
"(2050, 8757)",177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,665.556369,...,70.242942,61.256551,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734
"(2050, 8758)",167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,626.856424,...,63.957702,55.775401,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508


In [29]:
#Storing value in a new dataframe so that in case of error we don't need to create processed dataframe once again
df_copy = df_original_form

In [30]:
#Checking dataframe
df_copy

,0,1,2,3,4,5,6,7,8,9,...,3099,3100,3101,3102,3103,3104,3105,3106,3107,3108
,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,p53067,...,p23013,p23015,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031
"(2010, 0)",81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,306.63252,...,34.499333,30.08573,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801
"(2010, 1)",79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,296.673894,...,32.751723,28.561696,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638
"(2010, 2)",77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,288.959841,...,31.264158,27.26444,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752
"(2010, 3)",76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,284.751003,...,30.328845,26.448784,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2050, 8755)",252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,945.992247,...,79.135891,69.011798,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611
"(2050, 8756)",237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,888.628268,...,74.361368,64.848094,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201
"(2050, 8757)",177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,665.556369,...,70.242942,61.256551,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734
"(2050, 8758)",167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,626.856424,...,63.957702,55.775401,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508


In [31]:
# # Resetting column headers to make sure correct 1st row is treated as header
df_copy.columns = df_copy.iloc[0]
df_copy = df_copy.iloc[1:]

#df_copy.reset_index()

In [32]:
df_copy

,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,p53067,...,p23013,p23015,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031
"(2010, 0)",81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,306.63252,...,34.499333,30.08573,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801
"(2010, 1)",79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,296.673894,...,32.751723,28.561696,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638
"(2010, 2)",77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,288.959841,...,31.264158,27.26444,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752
"(2010, 3)",76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,284.751003,...,30.328845,26.448784,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398
"(2010, 4)",76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,802.582772,286.910294,...,30.069051,26.222227,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2050, 8755)",252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,945.992247,...,79.135891,69.011798,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611
"(2050, 8756)",237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,888.628268,...,74.361368,64.848094,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201
"(2050, 8757)",177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,665.556369,...,70.242942,61.256551,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734
"(2050, 8758)",167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,626.856424,...,63.957702,55.775401,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508


In [33]:
#Reset index to make the year, hour pair column as a real column to be split and added back to the dataframe
df_copy = df_copy.reset_index()

In [34]:
#Check operation completed successfully
df_copy

,index,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,...,p23013,p23015,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031
0,"(2010, 0)",81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,...,34.499333,30.08573,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801
1,"(2010, 1)",79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,...,32.751723,28.561696,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638
2,"(2010, 2)",77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,...,31.264158,27.26444,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752
3,"(2010, 3)",76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,...,30.328845,26.448784,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398
4,"(2010, 4)",76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,802.582772,...,30.069051,26.222227,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,"(2050, 8755)",252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,...,79.135891,69.011798,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611
359156,"(2050, 8756)",237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,...,74.361368,64.848094,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201
359157,"(2050, 8757)",177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,...,70.242942,61.256551,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734
359158,"(2050, 8758)",167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,...,63.957702,55.775401,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508


In [35]:
#Split data in column named index as it has year, hour pairs
df_copy[['year','hour']] = df_copy['index'].str.split(', ', expand=True)
#df_copy = df_copy.reset_index()
df_copy

,index,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,...,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031,year,hour
0,"(2010, 0)",81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,...,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801,(2010,0)
1,"(2010, 1)",79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,...,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638,(2010,1)
2,"(2010, 2)",77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,...,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752,(2010,2)
3,"(2010, 3)",76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,...,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398,(2010,3)
4,"(2010, 4)",76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,802.582772,...,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437,(2010,4)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,"(2050, 8755)",252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,...,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611,(2050,8755)
359156,"(2050, 8756)",237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,...,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201,(2050,8756)
359157,"(2050, 8757)",177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,...,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734,(2050,8757)
359158,"(2050, 8758)",167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,...,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508,(2050,8758)


In [36]:
#Clean up newly formed columns
df_copy['year'] = df_copy['year'].str.replace('(','')
df_copy['hour'] = df_copy['hour'].str.replace(')','')
df_copy

C:\Users\akarmaka\AppData\Local\Temp\27\ipykernel_18552\3012002316.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_copy['year'] = df_copy['year'].str.replace('(','')
C:\Users\akarmaka\AppData\Local\Temp\27\ipykernel_18552\3012002316.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_copy['hour'] = df_copy['hour'].str.replace(')','')


,index,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,...,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031,year,hour
0,"(2010, 0)",81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,...,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801,2010,0
1,"(2010, 1)",79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,...,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638,2010,1
2,"(2010, 2)",77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,...,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752,2010,2
3,"(2010, 3)",76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,...,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398,2010,3
4,"(2010, 4)",76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,802.582772,...,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437,2010,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,"(2050, 8755)",252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,...,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611,2050,8755
359156,"(2050, 8756)",237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,...,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201,2050,8756
359157,"(2050, 8757)",177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,...,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734,2050,8757
359158,"(2050, 8758)",167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,...,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508,2050,8758


In [37]:
#Drop column as it is no longer required anymore
df_copy.drop(['index'], axis=1, inplace=True)


df_copy

,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,p53067,...,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031,year,hour
0,81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,306.63252,...,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801,2010,0
1,79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,296.673894,...,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638,2010,1
2,77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,288.959841,...,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752,2010,2
3,76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,284.751003,...,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398,2010,3
4,76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,802.582772,286.910294,...,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437,2010,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,945.992247,...,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611,2050,8755
359156,237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,888.628268,...,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201,2050,8756
359157,177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,665.556369,...,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734,2050,8757
359158,167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,626.856424,...,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508,2050,8758


In [38]:
#Store dataframe into new variable - treat as checkpoint
df_final_new = df_copy
df_final_new

,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,p53067,...,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031,year,hour
0,81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,857.752346,306.63252,...,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801,2010,0
1,79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,829.894782,296.673894,...,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638,2010,1
2,77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,808.316031,288.959841,...,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752,2010,2
3,76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,796.542521,284.751003,...,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398,2010,3
4,76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,802.582772,286.910294,...,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437,2010,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,2646.252487,945.992247,...,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611,2050,8755
359156,237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,2485.786508,888.628268,...,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201,2050,8756
359157,177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,1861.780793,665.556369,...,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734,2050,8757
359158,167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,1753.524278,626.856424,...,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508,2050,8758


In [39]:
#Rename columns and create new ones at appropriate positions to redefine new index
df_final_new.rename(columns = {'year':'year_copy', 'hour':'hour_copy'}, inplace = True)

df_final_new.insert(0, 'year', True)
df_final_new.insert(1, 'hour', True)

df_final_new

,year,hour,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,...,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031,year_copy,hour_copy
0,True,True,81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,...,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801,2010,0
1,True,True,79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,...,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638,2010,1
2,True,True,77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,...,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752,2010,2
3,True,True,76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,...,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398,2010,3
4,True,True,76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,...,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437,2010,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,True,True,252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,...,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611,2050,8755
359156,True,True,237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,...,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201,2050,8756
359157,True,True,177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,...,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734,2050,8757
359158,True,True,167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,...,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508,2050,8758


In [40]:
#Copy data into blank new columns
df_final_new['year'] = df_final_new['year_copy']
df_final_new['hour'] = df_final_new['hour_copy']
df_final_new

,year,hour,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,...,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031,year_copy,hour_copy
0,2010,0,81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,...,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801,2010,0
1,2010,1,79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,...,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638,2010,1
2,2010,2,77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,...,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752,2010,2
3,2010,3,76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,...,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398,2010,3
4,2010,4,76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,...,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437,2010,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,2050,8755,252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,...,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611,2050,8755
359156,2050,8756,237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,...,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201,2050,8756
359157,2050,8757,177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,...,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734,2050,8757
359158,2050,8758,167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,...,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508,2050,8758


In [41]:
#Drop columns no  longer required
df_final_new.drop(['year_copy', 'hour_copy'], axis=1, inplace=True)
df_final_new

,year,hour,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,...,p23013,p23015,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031
0,2010,0,81.959526,89.971691,2317.729081,282.282167,46.820637,952.597606,19.096037,134.492407,...,34.499333,30.08573,49.232339,128.280855,14.413909,31.12951,42.483455,33.515174,26.133136,180.197801
1,2010,1,79.297694,87.049644,2242.455273,273.114376,45.300025,921.659716,18.475847,130.124443,...,32.751723,28.561696,46.738408,121.782615,13.683753,29.552603,40.331398,31.817417,24.809327,171.069638
2,2010,2,77.235812,84.786198,2184.147417,266.012913,44.122143,897.694911,17.995442,126.740975,...,31.264158,27.26444,44.615576,116.251318,13.062245,28.21034,38.499569,30.372288,23.682501,163.299752
3,2010,3,76.110835,83.551246,2152.334264,262.138308,43.479483,884.619555,17.73333,124.894932,...,30.328845,26.448784,43.280835,112.773488,12.671469,27.366386,37.347798,29.463656,22.974004,158.414398
4,2010,4,76.68799,84.184822,2168.655601,264.126125,43.809192,891.327701,17.867803,125.842021,...,30.069051,26.222227,42.910096,111.807482,12.562926,27.131969,37.02788,29.211274,22.777211,157.057437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359155,2050,8755,252.853403,277.571739,7150.427944,870.86895,144.446388,2938.85968,58.913198,414.922637,...,79.135891,69.011798,112.931023,294.255535,33.063177,71.406061,97.450175,76.878388,59.945187,413.344611
359156,2050,8756,237.520638,260.74008,6716.833483,818.060369,135.68731,2760.650307,55.340764,389.762163,...,74.361368,64.848094,106.117531,276.502152,31.068369,67.097904,91.570693,72.240067,56.328501,388.406201
359157,2050,8757,177.895953,195.286631,5030.710212,612.703094,101.625794,2067.645674,41.4486,291.920367,...,70.242942,61.256551,100.240324,261.188372,29.347681,63.381757,86.499147,68.239128,53.208807,366.894734
359158,2050,8758,167.551881,183.931347,4738.190727,577.076396,95.716584,1947.418783,39.038498,274.946144,...,63.957702,55.775401,91.27096,237.817603,26.721691,57.710447,78.759324,62.133186,48.447757,334.065508


In [42]:
#Check datatypes
df_final_new.dtypes


year      object
hour      object
p53007    object
p53029    object
p53033    object
           ...  
p23023    object
p23025    object
p23027    object
p23029    object
p23031    object
Length: 3111, dtype: object

In [43]:
df_final_new['year']=df_final_new['year'].apply(pd.to_numeric)
df_final_new['hour']=df_final_new['hour'].apply(pd.to_numeric)
df_final_new.dtypes


year       int64
hour       int64
p53007    object
p53029    object
p53033    object
           ...  
p23023    object
p23025    object
p23027    object
p23029    object
p23031    object
Length: 3111, dtype: object

In [44]:
#Set the new index multidimensional as year, hour as in original file
df_final_new = df_final_new.set_index(['year','hour'])
df_final_new

p53007      p53029       p53033      p53035      p53037  \
year hour                                                                
2010 0      81.959526   89.971691  2317.729081  282.282167   46.820637   
     1      79.297694   87.049644  2242.455273  273.114376   45.300025   
     2      77.235812   84.786198  2184.147417  266.012913   44.122143   
     3      76.110835   83.551246  2152.334264  262.138308   43.479483   
     4       76.68799   84.184822  2168.655601  264.126125   43.809192   
...               ...         ...          ...         ...         ...   
2050 8755  252.853403  277.571739  7150.427944   870.86895  144.446388   
     8756  237.520638   260.74008  6716.833483  818.060369   135.68731   
     8757  177.895953  195.286631  5030.710212  612.703094  101.625794   
     8758  167.551881  183.931347  4738.190727  577.076396   95.716584   
     8759  159.280603  174.851489  4504.287696  548.588746   90.991489   

                p53053     p53055      p53057       p53061      p53067  ...  \
year hour                                                               ...   
2010 0      952.597606  19.096037  134.492407   857.752346   306.63252  ...   
     1      921.659716  18.475847  130.124443   829.894782  296.673894  ...   
     2      897.694911  17.995442  126.740975   808.316031  288.959841  ...   
     3      884.619555   17.73333  124.894932   796.542521  284.751003  ...   
     4      891.327701  17.867803  125.842021   802.582772  286.910294  ...   
...                ...        ...         ...          ...         ...  ...   
2050 8755   2938.85968  58.913198  414.922637  2646.252487  945.992247  ...   
     8756  2760.650307  55.340764  389.762163  2485.786508  888.628268  ...   
     8757  2067.645674    41.4486  291.920367  1861.780793  665.556369  ...   
     8758  1947.418783  39.038498  274.946144  1753.524278  626.856424  ...   
     8759  1851.283532  37.111345  261.373297  1666.960721   595.91136  ...   

              p23013     p23015      p23017      p23019     p23021     p23023  \
year hour                                                                       
2010 0     34.499333   30.08573   49.232339  128.280855  14.413909   31.12951   
     1     32.751723  28.561696   46.738408  121.782615  13.683753  29.552603   
     2     31.264158   27.26444   44.615576  116.251318  13.062245   28.21034   
     3     30.328845  26.448784   43.280835  112.773488  12.671469  27.366386   
     4     30.069051  26.222227   42.910096  111.807482  12.562926  27.131969   
...              ...        ...         ...         ...        ...        ...   
2050 8755  79.135891  69.011798  112.931023  294.255535  33.063177  71.406061   
     8756  74.361368  64.848094  106.117531  276.502152  31.068369  67.097904   
     8757  70.242942  61.256551  100.240324  261.188372  29.347681  63.381757   
     8758  63.957702  55.775401    91.27096  237.817603  26.721691  57.710447   
     8759    57.4004  50.056994   81.913349  213.435208  23.982034  51.793648   

              p23025     p23027     p23029      p23031  
year hour                                               
2010 0     42.483455  33.515174  26.133136  180.197801  
     1     40.331398  31.817417  24.809327  171.069638  
     2     38.499569  30.372288  23.682501  163.299752  
     3     37.347798  29.463656  22.974004  158.414398  
     4      37.02788  29.211274  22.777211  157.057437  
...              ...        ...        ...         ...  
2050 8755  97.450175  76.878388  59.945187  413.344611  
     8756  91.570693  72.240067  56.328501  388.406201  
     8757  86.499147  68.239128  53.208807  366.894734  
     8758  78.759324  62.133186  48.447757  334.065508  
     8759  70.684477  55.762943  43.480621  299.815238  

[359160 rows x 3109 columns]

In [45]:
#Check datatypes
df_final_new.dtypes


p53007    object
p53029    object
p53033    object
p53035    object
p53037    object
           ...  
p23023    object
p23025    object
p23027    object
p23029    object
p23031    object
Length: 3109, dtype: object

In [46]:
#Convert to float otherwise python has a hard time writing the file
df_final_new_float = df_final_new.astype('float64', copy=False)
df_final_new_float

p53007      p53029       p53033      p53035      p53037  \
year hour                                                                
2010 0      81.959526   89.971691  2317.729081  282.282167   46.820637   
     1      79.297694   87.049644  2242.455273  273.114376   45.300025   
     2      77.235812   84.786198  2184.147417  266.012913   44.122143   
     3      76.110835   83.551246  2152.334264  262.138308   43.479483   
     4      76.687990   84.184822  2168.655601  264.126125   43.809192   
...               ...         ...          ...         ...         ...   
2050 8755  252.853403  277.571739  7150.427944  870.868950  144.446388   
     8756  237.520638  260.740080  6716.833483  818.060369  135.687310   
     8757  177.895953  195.286631  5030.710212  612.703094  101.625794   
     8758  167.551881  183.931347  4738.190727  577.076396   95.716584   
     8759  159.280603  174.851489  4504.287696  548.588746   90.991489   

                p53053     p53055      p53057       p53061      p53067  ...  \
year hour                                                               ...   
2010 0      952.597606  19.096037  134.492407   857.752346  306.632520  ...   
     1      921.659716  18.475847  130.124443   829.894782  296.673894  ...   
     2      897.694911  17.995442  126.740975   808.316031  288.959841  ...   
     3      884.619555  17.733330  124.894932   796.542521  284.751003  ...   
     4      891.327701  17.867803  125.842021   802.582772  286.910294  ...   
...                ...        ...         ...          ...         ...  ...   
2050 8755  2938.859680  58.913198  414.922637  2646.252487  945.992247  ...   
     8756  2760.650307  55.340764  389.762163  2485.786508  888.628268  ...   
     8757  2067.645674  41.448600  291.920367  1861.780793  665.556369  ...   
     8758  1947.418783  39.038498  274.946144  1753.524278  626.856424  ...   
     8759  1851.283532  37.111345  261.373297  1666.960721  595.911360  ...   

              p23013     p23015      p23017      p23019     p23021     p23023  \
year hour                                                                       
2010 0     34.499333  30.085730   49.232339  128.280855  14.413909  31.129510   
     1     32.751723  28.561696   46.738408  121.782615  13.683753  29.552603   
     2     31.264158  27.264440   44.615576  116.251318  13.062245  28.210340   
     3     30.328845  26.448784   43.280835  112.773488  12.671469  27.366386   
     4     30.069051  26.222227   42.910096  111.807482  12.562926  27.131969   
...              ...        ...         ...         ...        ...        ...   
2050 8755  79.135891  69.011798  112.931023  294.255535  33.063177  71.406061   
     8756  74.361368  64.848094  106.117531  276.502152  31.068369  67.097904   
     8757  70.242942  61.256551  100.240324  261.188372  29.347681  63.381757   
     8758  63.957702  55.775401   91.270960  237.817603  26.721691  57.710447   
     8759  57.400400  50.056994   81.913349  213.435208  23.982034  51.793648   

              p23025     p23027     p23029      p23031  
year hour                                               
2010 0     42.483455  33.515174  26.133136  180.197801  
     1     40.331398  31.817417  24.809327  171.069638  
     2     38.499569  30.372288  23.682501  163.299752  
     3     37.347798  29.463656  22.974004  158.414398  
     4     37.027880  29.211274  22.777211  157.057437  
...              ...        ...        ...         ...  
2050 8755  97.450175  76.878388  59.945187  413.344611  
     8756  91.570693  72.240067  56.328501  388.406201  
     8757  86.499147  68.239128  53.208807  366.894734  
     8758  78.759324  62.133186  48.447757  334.065508  
     8759  70.684477  55.762943  43.480621  299.815238  

[359160 rows x 3109 columns]

In [60]:
df_final_new_float_rounded = df_final_new_float.round(decimals=2)
df_final_new_float_rounded.sample()

,,p53007,p53029,p53033,p53035,p53037,p53053,p53055,p53057,p53061,p53067,...,p23013,p23015,p23017,p23019,p23021,p23023,p23025,p23027,p23029,p23031
year,hour,,,,,,,,,,,,,,,,,,,,,
2019,6321,84.67,92.94,2394.3,291.61,48.37,984.07,19.73,138.94,886.09,316.76,...,38.95,33.96,55.58,144.81,16.27,35.14,47.96,37.83,29.5,203.42


In [47]:
# df_final_new_float.dtypes


p53007    float64
p53029    float64
p53033    float64
p53035    float64
p53037    float64
           ...   
p23023    float64
p23025    float64
p23027    float64
p23029    float64
p23031    float64
Length: 3109, dtype: object

In [61]:
df_final_new_float_rounded

p53007  p53029   p53033  p53035  p53037   p53053  p53055  p53057  \
year hour                                                                     
2010 0      81.96   89.97  2317.73  282.28   46.82   952.60   19.10  134.49   
     1      79.30   87.05  2242.46  273.11   45.30   921.66   18.48  130.12   
     2      77.24   84.79  2184.15  266.01   44.12   897.69   18.00  126.74   
     3      76.11   83.55  2152.33  262.14   43.48   884.62   17.73  124.89   
     4      76.69   84.18  2168.66  264.13   43.81   891.33   17.87  125.84   
...           ...     ...      ...     ...     ...      ...     ...     ...   
2050 8755  252.85  277.57  7150.43  870.87  144.45  2938.86   58.91  414.92   
     8756  237.52  260.74  6716.83  818.06  135.69  2760.65   55.34  389.76   
     8757  177.90  195.29  5030.71  612.70  101.63  2067.65   41.45  291.92   
     8758  167.55  183.93  4738.19  577.08   95.72  1947.42   39.04  274.95   
     8759  159.28  174.85  4504.29  548.59   90.99  1851.28   37.11  261.37   

            p53061  p53067  ...  p23013  p23015  p23017  p23019  p23021  \
year hour                   ...                                           
2010 0      857.75  306.63  ...   34.50   30.09   49.23  128.28   14.41   
     1      829.89  296.67  ...   32.75   28.56   46.74  121.78   13.68   
     2      808.32  288.96  ...   31.26   27.26   44.62  116.25   13.06   
     3      796.54  284.75  ...   30.33   26.45   43.28  112.77   12.67   
     4      802.58  286.91  ...   30.07   26.22   42.91  111.81   12.56   
...            ...     ...  ...     ...     ...     ...     ...     ...   
2050 8755  2646.25  945.99  ...   79.14   69.01  112.93  294.26   33.06   
     8756  2485.79  888.63  ...   74.36   64.85  106.12  276.50   31.07   
     8757  1861.78  665.56  ...   70.24   61.26  100.24  261.19   29.35   
     8758  1753.52  626.86  ...   63.96   55.78   91.27  237.82   26.72   
     8759  1666.96  595.91  ...   57.40   50.06   81.91  213.44   23.98   

           p23023  p23025  p23027  p23029  p23031  
year hour                                          
2010 0      31.13   42.48   33.52   26.13  180.20  
     1      29.55   40.33   31.82   24.81  171.07  
     2      28.21   38.50   30.37   23.68  163.30  
     3      27.37   37.35   29.46   22.97  158.41  
     4      27.13   37.03   29.21   22.78  157.06  
...           ...     ...     ...     ...     ...  
2050 8755   71.41   97.45   76.88   59.95  413.34  
     8756   67.10   91.57   72.24   56.33  388.41  
     8757   63.38   86.50   68.24   53.21  366.89  
     8758   57.71   78.76   62.13   48.45  334.07  
     8759   51.79   70.68   55.76   43.48  299.82  

[359160 rows x 3109 columns]

In [59]:
# for year in df_final_new_float.index.get_level_values('year').unique():
#     df_year = df_final_new_float.loc[year]#df_final_new_float[df_final_new_float['year'] == year]
    
#     df_year.to_hdf(f'D://akash//county_data//h5_county_EPMEDIUM_load_hourly_{year}.h5', key='data', complevel=9, complib='zlib', index=True, encoding = 'utf-8')


In [62]:
for year in df_final_new_float_rounded.index.get_level_values('year').unique():
    df_year = df_final_new_float_rounded.loc[year]#df_final_new_float[df_final_new_float['year'] == year]
    
    df_year.to_hdf(f'D://akash//county_data//h5_county_EPMEDIUM_load_hourly_{year}.h5', key='data', complevel=9, complib='zlib', index=True, encoding = 'utf-8')


In [ ]:
for year in df_final_new_float_rounded.index.get_level_values('year').unique():
    df_year = df_final_new_float_rounded.loc[year]#df_final_new_float[df_final_new_float['year'] == year]
    
    df_year.to_feather(f'D://akash//county_data//h5_county_EPMEDIUM_load_hourly_{year}.feather', compression='gzip', index=True, encoding = 'utf-8')


In [48]:
# #Write processed county load dataframe into an h5 file
# df_final_new_float.to_hdf('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_v2.h5', key='data', complevel=9, complib='zlib', index=True, encoding = 'utf-8')


In [49]:
# # different compressions
# #Write processed county load dataframe into an h5 file
# df_final_new_float.to_hdf('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_v2_lzo.h5', key='data', complevel=9, complib='lzo', index=True, encoding = 'utf-8')


In [50]:
# #Trial 
# #Write processed county load dataframe into an h5 file
# df_final_new_float.to_hdf('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_v2_bzip2.h5', key='data', complevel=9, complib='bzip2', index=True, encoding = 'utf-8')


In [51]:
# #Trial 
# #Write processed county load dataframe into an h5 file
# df_final_new_float.to_hdf('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_v2_blosc.h5', key='data', complevel=9, complib='blosc', index=True, encoding = 'utf-8')


In [54]:
# #Trial 
# #Write processed county load dataframe into an parquet file
# df_final_new_float.to_parquet('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_v2_parquet.parquet', engine='auto', compression = 'snappy', index=True)


In [ ]:
# #Read the written file to ensire it reads correctly again
# def read_file(filename, index_columns=1):
#     """
#     Read input file of various types (for backwards-compatibility)
#     """
#     # Try reading a .h5 file written by pandas
#     try:
#         df = pd.read_hdf(filename+'.h5')
#     # Try reading a .h5 file written by h5py
#     except (ValueError, TypeError, FileNotFoundError, OSError):
#         try:
#             with h5py.File(filename+'.h5', 'r') as f:
#                 keys = list(f)
#                 datakey = 'data' if 'data' in keys else ('cf' if 'cf' in keys else 'load')
#                 ### If none of these keys work, we're dealing with EER-formatted load
#                 if datakey not in keys:
#                     years = [int(y) for y in keys if y != 'columns']
#                     df = pd.concat(
#                         {y: pd.DataFrame(f[str(y)][...]) for y in years},
#                         axis=0)
#                     df.index = df.index.rename(['year','hour'])
#                 else:
#                     df = pd.DataFrame(f[datakey][:])
#                     df.index = pd.Series(f['index']).values
#                 df.columns = pd.Series(f['columns']).map(
#                     lambda x: x if type(x) is str else x.decode('utf-8')).values
#         # Fall back to .csv.gz
#         except (FileNotFoundError, OSError):
#             df = pd.read_csv(
#                 filename+'.csv.gz', index_col=list(range(index_columns)),
#                 float_precision='round_trip',
#             )

#     return df


# df_temp_check = read_file('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_v2',
#             index_columns=2)
# df_temp_check

In [58]:
#Read the written file to ensire it reads correctly again
def read_file(filename, index_columns=1):
    """
    Read input file of various types (for backwards-compatibility)
    """
    # Try reading a .h5 file written by pandas
    try:
        df = pd.read_hdf(filename+'.h5')
    # Try reading a .h5 file written by h5py
    except (ValueError, TypeError, FileNotFoundError, OSError):
        try:
            with h5py.File(filename+'.h5', 'r') as f:
                keys = list(f)
                datakey = 'data' if 'data' in keys else ('cf' if 'cf' in keys else 'load')
                ### If none of these keys work, we're dealing with EER-formatted load
                if datakey not in keys:
                    years = [int(y) for y in keys if y != 'columns']
                    df = pd.concat(
                        {y: pd.DataFrame(f[str(y)][...]) for y in years},
                        axis=0)
                    df.index = df.index.rename(['year','hour'])
                else:
                    df = pd.DataFrame(f[datakey][:])
                    df.index = pd.Series(f['index']).values
                df.columns = pd.Series(f['columns']).map(
                    lambda x: x if type(x) is str else x.decode('utf-8')).values
        # Fall back to .csv.gz
        except (FileNotFoundError, OSError):
            df = pd.read_csv(
                filename+'.csv.gz', index_col=list(range(index_columns)),
                float_precision='round_trip',
            )

    return df


df_temp_check = read_file('D://akash//county_data//h5_county_EPMEDIUM_load_hourly_2010',
            index_columns=2)
df_temp_check

p53007      p53029       p53033      p53035      p53037  \
year hour                                                                
2010 0      81.959526   89.971691  2317.729081  282.282167   46.820637   
     1      79.297694   87.049644  2242.455273  273.114376   45.300025   
     2      77.235812   84.786198  2184.147417  266.012913   44.122143   
     3      76.110835   83.551246  2152.334264  262.138308   43.479483   
     4      76.687990   84.184822  2168.655601  264.126125   43.809192   
...               ...         ...          ...         ...         ...   
2050 8755  252.853403  277.571739  7150.427944  870.868950  144.446388   
     8756  237.520638  260.740080  6716.833483  818.060369  135.687310   
     8757  177.895953  195.286631  5030.710212  612.703094  101.625794   
     8758  167.551881  183.931347  4738.190727  577.076396   95.716584   
     8759  159.280603  174.851489  4504.287696  548.588746   90.991489   

                p53053     p53055      p53057       p53061      p53067  ...  \
year hour                                                               ...   
2010 0      952.597606  19.096037  134.492407   857.752346  306.632520  ...   
     1      921.659716  18.475847  130.124443   829.894782  296.673894  ...   
     2      897.694911  17.995442  126.740975   808.316031  288.959841  ...   
     3      884.619555  17.733330  124.894932   796.542521  284.751003  ...   
     4      891.327701  17.867803  125.842021   802.582772  286.910294  ...   
...                ...        ...         ...          ...         ...  ...   
2050 8755  2938.859680  58.913198  414.922637  2646.252487  945.992247  ...   
     8756  2760.650307  55.340764  389.762163  2485.786508  888.628268  ...   
     8757  2067.645674  41.448600  291.920367  1861.780793  665.556369  ...   
     8758  1947.418783  39.038498  274.946144  1753.524278  626.856424  ...   
     8759  1851.283532  37.111345  261.373297  1666.960721  595.911360  ...   

              p23013     p23015      p23017      p23019     p23021     p23023  \
year hour                                                                       
2010 0     34.499333  30.085730   49.232339  128.280855  14.413909  31.129510   
     1     32.751723  28.561696   46.738408  121.782615  13.683753  29.552603   
     2     31.264158  27.264440   44.615576  116.251318  13.062245  28.210340   
     3     30.328845  26.448784   43.280835  112.773488  12.671469  27.366386   
     4     30.069051  26.222227   42.910096  111.807482  12.562926  27.131969   
...              ...        ...         ...         ...        ...        ...   
2050 8755  79.135891  69.011798  112.931023  294.255535  33.063177  71.406061   
     8756  74.361368  64.848094  106.117531  276.502152  31.068369  67.097904   
     8757  70.242942  61.256551  100.240324  261.188372  29.347681  63.381757   
     8758  63.957702  55.775401   91.270960  237.817603  26.721691  57.710447   
     8759  57.400400  50.056994   81.913349  213.435208  23.982034  51.793648   

              p23025     p23027     p23029      p23031  
year hour                                               
2010 0     42.483455  33.515174  26.133136  180.197801  
     1     40.331398  31.817417  24.809327  171.069638  
     2     38.499569  30.372288  23.682501  163.299752  
     3     37.347798  29.463656  22.974004  158.414398  
     4     37.027880  29.211274  22.777211  157.057437  
...              ...        ...        ...         ...  
2050 8755  97.450175  76.878388  59.945187  413.344611  
     8756  91.570693  72.240067  56.328501  388.406201  
     8757  86.499147  68.239128  53.208807  366.894734  
     8758  78.759324  62.133186  48.447757  334.065508  
     8759  70.684477  55.762943  43.480621  299.815238  

[359160 rows x 3109 columns]